In [314]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
import category_encoders as ce
import functools

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Conv1D, Embedding,Reshape, Concatenate,BatchNormalization,GlobalMaxPooling1D,GlobalAveragePooling1D
from keras.layers.merge import Concatenate
from keras.preprocessing import sequence
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras import metrics
from keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction import FeatureHasher

# Lectura de datos

In [315]:
data = pd.read_csv('C:/Users/Ceiec04/Desktop/Prediccion_Alertas_Alimentarias/Datasets/splited_RASFF_DATA_16092019.csv', sep=';', 
                header=0, index_col = 0)
data ['DATE_CASE'] = data ['DATE_CASE'].astype(str)
data ['HAZARDS_CAT'] = data ['HAZARDS_CAT'].astype(str)
data = data.sample(frac = 1)
df = data
df

,DATE_CASE,NOT_COUNTRY,PROD_CAT,TYPE,RISK_DECISION,ACTION_TAKEN,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN,COUNT_DESTIN,COUNT_CONCERN
38473,2017-01-12,Germany,cephalopods and products thereof,food,serious,seizure,distribution to other member countries,metals,India,Sweden,INFOSAN
81393,2012-09-07,Netherlands,herbs and spices,food,serious,re-dispatch,information on distribution not (yet) available,pathogenic micro-organisms,Germany,Germany,Commission Services
98111,2010-10-27,Netherlands,"nuts, nut products and seeds",food,undecided,(obsolete),no distribution,mycotoxins,Brazil,,Netherlands
104115,2009-12-22,Netherlands,"nuts, nut products and seeds",food,undecided,official detention,no distribution,mycotoxins,Egypt,,Netherlands
107327,2009-08-03,Italy,cereals and bakery products,food,undecided,seizure,distribution on the market (possible),foreign bodies,France,Italy,
131374,2005-11-09,Slovenia,animal nutrition - (obsolete),feed,undecided,,distribution on the market (possible),pathogenic micro-organisms,Bosnia and Herzegovina,Italy,Spain
114796,2008-08-11,Italy,fish and fish products,food,undecided,re-dispatch,no distribution,poor or insufficient controls,Croatia,,Italy
73701,2013-07-09,United Kingdom,fruits and vegetables,food,serious,informing authorities,product (presumably) no longer on the market,composition,Costa Rica,United Kingdom,
93018,2011-06-10,Poland,cereals and bakery products,food,undecided,re-dispatch,no distribution,foreign bodies,Ukraine,,Poland
66580,2014-04-02,France,"dietetic foods, food supplements, fortified foods",food,serious,recall from consumers,distribution to other member countries,composition,United Kingdom,Ireland,Netherlands


In [316]:
mask = (df['DATE_CASE'] >= '2004-01-01') & (df['DATE_CASE'] <= '2018-12-31')
df = df.loc[mask]
df['DATE_CASE'] =pd.to_datetime(df.DATE_CASE)
df['DATE_CASE'] = df.DATE_CASE.dt.month

In [318]:
features = [0,1,2,6,7,8]
target = [5]
X = df.iloc[:,features]
y = df.iloc[:,target]

In [319]:
X

,DATE_CASE,NOT_COUNTRY,PROD_CAT,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN
38473,1,Germany,cephalopods and products thereof,distribution to other member countries,metals,India
81393,9,Netherlands,herbs and spices,information on distribution not (yet) available,pathogenic micro-organisms,Germany
98111,10,Netherlands,"nuts, nut products and seeds",no distribution,mycotoxins,Brazil
104115,12,Netherlands,"nuts, nut products and seeds",no distribution,mycotoxins,Egypt
107327,8,Italy,cereals and bakery products,distribution on the market (possible),foreign bodies,France
131374,11,Slovenia,animal nutrition - (obsolete),distribution on the market (possible),pathogenic micro-organisms,Bosnia and Herzegovina
114796,8,Italy,fish and fish products,no distribution,poor or insufficient controls,Croatia
73701,7,United Kingdom,fruits and vegetables,product (presumably) no longer on the market,composition,Costa Rica
93018,6,Poland,cereals and bakery products,no distribution,foreign bodies,Ukraine
66580,4,France,"dietetic foods, food supplements, fortified foods",distribution to other member countries,composition,United Kingdom


In [320]:
y

,ACTION_TAKEN
38473,seizure
81393,re-dispatch
98111,(obsolete)
104115,official detention
107327,seizure
131374,
114796,re-dispatch
73701,informing authorities
93018,re-dispatch
66580,recall from consumers


# Selección de codificación

In [321]:
ency = OneHotEncoder(handle_unknown='ignore', sparse = False)
ency.fit(y.values)
y_one_hot = ency.transform(y.values)

In [322]:
'''encx = OrdinalEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

'encx = OrdinalEncoder()\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)'

In [323]:
'''encx = OneHotEncoder(handle_unknown='ignore', sparse = False)
encx.fit(X.values)
x_one_hot = encx.transform(X.values)'''

"encx = OneHotEncoder(handle_unknown='ignore', sparse = False)\nencx.fit(X.values)\nx_one_hot = encx.transform(X.values)"

In [324]:
encx  = ce.BaseNEncoder()
encx.fit(X.values)
x_one_hot = encx.transform(X.values)

In [326]:
X_train, X_val, y_train, y_val = train_test_split(x_one_hot, y_one_hot, test_size=0.2)

# Pruebas con mlp básico

In [329]:
accs = []
for i in range (1,6):
    K.clear_session()
    model = Sequential()
    model.add(Dense(2048, activation="relu", input_dim=len(X_train.values[1])))
    model.add(Dropout(0.3))
    model.add(Dense(1024, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(24, activation = "softmax"))
    model.compile(loss='categorical_crossentropy', optimizer="adam",  metrics=['categorical_accuracy'])
    hist = model.fit(X_train,y_train, epochs= 15,validation_data=(X_val, y_val), batch_size = 64)
    accs.append(hist.history['val_categorical_accuracy'][9])
    model.save ("model"+str(i)+".h5")

Train on 105355 samples, validate on 26339 samples
Epoch 1/15
105355/105355 [==============================] - 8s 72us/step - loss: 1.7786 - categorical_accuracy: 0.4564 - val_loss: 1.4485 - val_categorical_accuracy: 0.5461
Epoch 2/15
105355/105355 [==============================] - 7s 68us/step - loss: 1.3853 - categorical_accuracy: 0.5702 - val_loss: 1.2360 - val_categorical_accuracy: 0.6137
Epoch 3/15
105355/105355 [==============================] - 7s 68us/step - loss: 1.2092 - categorical_accuracy: 0.6202 - val_loss: 1.1074 - val_categorical_accuracy: 0.6480
Epoch 4/15
105355/105355 [==============================] - 7s 68us/step - loss: 1.0989 - categorical_accuracy: 0.6521 - val_loss: 1.0380 - val_categorical_accuracy: 0.6712
Epoch 5/15
105355/105355 [==============================] - 7s 69us/step - loss: 1.0197 - categorical_accuracy: 0.6716 - val_loss: 0.9857 - val_categorical_accuracy: 0.6860
Epoch 6/15
105355/105355 [==============================] - 7s 68us/step - loss: 0.9

In [349]:
suma = 0
for i in accs:
    suma = suma + i
print(suma/5)

0.7319564144768591
